### Create Workspace

In [3]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

1.0.39


In [5]:
ws = Workspace.create(name='prodmod',
                      subscription_id='3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d',
                      resource_group='prodmod_rg',
                      create_resource_group=True,
                      location='westus2' 
                     )

Deploying KeyVault with name prodmodkeyvault4dffdcce4.
Deploying StorageAccount with name prodmodstorage3dd946b962.
Deploying AppInsights with name prodmodinsightscc2312924.
Deployed AppInsights with name prodmodinsightscc2312924.
Deployed KeyVault with name prodmodkeyvault4dffdcce4.
Deploying Workspace with name prodmod.
Deployed StorageAccount with name prodmodstorage3dd946b962.
Deployed Workspace with name prodmod.


In [6]:
ws.get_details()

{'id': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourceGroups/prodmod_rg/providers/Microsoft.MachineLearningServices/workspaces/prodmod',
 'name': 'prodmod',
 'location': 'westus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '60b5573d-c2c7-4ff1-bd7b-b0d4966cdc90',
 'description': '',
 'friendlyName': 'prodmod',
 'creationTime': '2019-05-20T21:03:06.5153700+00:00',
 'keyVault': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourcegroups/prodmod_rg/providers/microsoft.keyvault/vaults/prodmodkeyvault4dffdcce4',
 'applicationInsights': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourcegroups/prodmod_rg/providers/microsoft.insights/components/prodmodinsightscc2312924',
 'identityPrincipalId': '7c45ae4c-f97c-4e6b-ba0d-446883f98526',
 'identityTenantId': 'bc63996d-bc1e-4645-8837-ef126b77b846',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourcegroups/prodmod_rg/provide

In [7]:
ws.write_config()

### Connect to workspace

In [8]:
ws = Workspace.from_config()

In [9]:
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

prodmod	westus2	prodmod_rg	westus2


### Create an experiment

In [10]:
experiment_name = 'prodmod'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Create Compute

In [11]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

In [14]:
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "prodmod-compute1")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

In [15]:
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes,
                                                                vm_priority = 'lowpriority',
                                                                idle_seconds_before_scaledown=600
                                                               )
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion()
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print(compute_target.get_status().serialize())

creating a new compute target...
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-05-20T21:20:35.692000+00:00', 'errors': None, 'creationTime': '2019-05-20T21:19:08.973390+00:00', 'modifiedTime': '2019-05-20T21:20:48.008942+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}
